# CNN

https://pytorch.org/docs/stable/nn.html?highlight=maxpool2d#torch.nn.MaxPool2d

https://pytorch.org/docs/stable/nn.html?highlight=conv2d#torch.nn.Conv2d


## basic example

In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)  # kernel size
        self.fc = nn.Linear(320, 10)  # avoid size mismatch, in: 64 * 320

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))  # max pool, print x.size() to check out
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        # It is almost always you will need the last dimension when you compute the cross-entropy
        return F.log_softmax(x, -1)  # add dim -1 to avoid warning...


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


# https://discuss.pytorch.org/t/two-small-questions-about-with-torch-no-grad/27571
@torch.no_grad()
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)  # change Variable(data, volatile=True)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.059847
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.194853
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.057874
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.083139
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.073367

Test set: Average loss: 0.0939, Accuracy: 9730/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.129999
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.144854
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.191636
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.069492
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.155353

Test set: Average loss: 0.0745, Accuracy: 9758/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.132020
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.071940
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.172060
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.129315
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.113893

Test set: Average loss: 0.0726, Accuracy: 9769/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.022187
Train Epoch: 4 [12800/60000 (21%)]	Lo

## advance example

In the context of convolutional neural networks, kernel = filter = feature detector

http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/

https://www.youtube.com/watch?v=VxhSouuSZDY

https://buzzrobot.com/whats-happening-inside-the-convolutional-neural-network-the-answer-is-convolution-2c22075dc68d

https://docs.python.org/3/library/argparse.html

In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse  # interesting package indeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
# make comparison of each convolution
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)


    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)


    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)  # inceptions
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        # Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
        return F.log_softmax(x, -1)


model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))


@torch.no_grad()
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.321258
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.051745
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.210887
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.033756
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.062098


/home/jian/miniconda3/envs/ptgpu/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0895, Accuracy: 9714/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.081887
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.198613
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.122356
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.120923
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.033388

Test set: Average loss: 0.0791, Accuracy: 9754/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.055966
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.103866
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.036902
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.048040
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.288322

Test set: Average loss: 0.0634, Accuracy: 9800/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.038738
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.077438
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.071596
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.026354
